In [1]:
import cv2
import torch
import yaml
from pathlib import Path
import ultralytics
from ultralytics import YOLO

In [2]:
green = {'short sleeve top', 'long sleeve top', 'short sleeve outerwear', 
                 'long sleeve outerwear', 'trousers', 'skirt', 'short sleeve dress', 
                 'long sleeve dress'}

In [ ]:
# Load YOLOv8 model
# Change this with finished model
model_path = "D:/Skripsi/Results/9-11-2023-50epch/weights/best.pt"
model = YOLO(model_path)

# Load YAML
args_path = "D:/Skripsi/Results/9-11-2023-50epch/args.yaml"
with open(args_path, 'r') as f:
    config = yaml.safe_load(f)

# Initialize the webcam using CV2
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Loop to continuously get frames from the webcam
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Perform inference & extract
    results = model(frame, verbose=False) #verbose=False so it doesn't
    boxes_data = results[0].boxes.data
    
    boxes_data_np = boxes_data.cpu().numpy()

    if boxes_data_np is not None:
            for box_data in boxes_data_np:
                x1, y1, x2, y2, conf, cls = box_data  # Unpack the bounding box information
                class_name = results[0].names[int(cls)]
                label = f"{class_name} {conf:.2f}"

                # Check if the detected class is in the set of green classes
                if class_name in green:
                    box_color = (0, 255, 0)
                else:
                    box_color = (0, 0, 255)

                # Draw bounding box and show category/confidence
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)
                (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
                cv2.rectangle(frame, (int(x1), int(y1) - text_height - 10), (int(x1) + text_width, int(y1) - 10), box_color, -1)
                cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, lineType=cv2.LINE_AA)
            
# Release resources
cap.release()
cv2.destroyAllWindows()